In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import torch
import numpy as np

In [4]:
torch.__version__

'2.0.0'

全結合層の定義

In [5]:
import torch.nn as nn

fully_connected_layer = nn.Linear(3, 2)
fully_connected_layer.weight


Parameter containing:
tensor([[ 0.0722, -0.0527,  0.0838],
        [ 0.3957, -0.1869, -0.1136]], requires_grad=True)

In [6]:
fully_connected_layer.bias

Parameter containing:
tensor([ 0.4593, -0.2142], requires_grad=True)

In [7]:
# seed value fixed for reproducibility
torch.manual_seed(1)

In [8]:
fully_connected_layer = nn.Linear(3, 2)

print("weight: ", fully_connected_layer.weight)
print("bias: ", fully_connected_layer.bias)

weight:  Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462]], requires_grad=True)
bias:  Parameter containing:
tensor([-0.1188,  0.2937], requires_grad=True)


In [9]:
x = torch.tensor([[1, 2, 3]], dtype=torch.float32) # float32 is the default (これにしないとエラーになる)

print("x: ", x)
print("x shape: ", x.shape)
print("x type: ", x.dtype)

x:  tensor([[1., 2., 3.]])
x shape:  torch.Size([1, 3])
x type:  torch.float32


In [10]:
# 線形変換
u = fully_connected_layer(x)

In [11]:
u

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward0>)

非線形関数

PyTorchで使用される関数はすべてtorch.nn.functionsに含まれている。

In [12]:
import torch.nn.functional as F

# 活性化関数
# ReLU関数
h = F.relu(u)
h

tensor([[0.0000, 0.5164]], grad_fn=<ReluBackward0>)

In [13]:
# seed value fixed for reproducibility
torch.manual_seed(1)

# input values
x = torch.tensor([[1, 2, 3]], dtype=torch.float32)
x

tensor([[1., 2., 3.]])

In [14]:
# fully connected layer
fc1 = nn.Linear(3, 2)
fc2 = nn.Linear(2, 1)

# 線形変換
u1 = fc1(x)
u1

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward0>)

In [15]:
# 非線形変換
h1 = F.relu(u1)

In [16]:
# 線形変換
y = fc2(h1)
y

tensor([[0.1514]], grad_fn=<AddmmBackward0>)

### 損失関数

今回は、損失関数として、二乗誤差を使用する。

In [17]:
# 目標値
t = torch.tensor([[1]], dtype=torch.float32)
t

tensor([[1.]])

In [18]:
# MSE: Mean Squared Error
loss = F.mse_loss(y, t)
loss

tensor(0.7201, grad_fn=<MseLossBackward0>)

### 学習の一連の流れ

- Step1: データの準備
- Step2: ネットワークの定義
- Step3: 損失関数の定義
- Step4: 最適化手法の定義
- Step5: 学習の実行

Scikit-learnのアヤメデータセットを使用する。

In [19]:
from sklearn.datasets import load_iris

# Irisデータセットの読み込み
x, t = load_iris(return_X_y=True)

# データの確認
x.shape, t.shape # type: ignore

((150, 4), (150,))

In [20]:
# check the first 5 rows of the data
x[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [21]:
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [22]:
x.dtype, t.dtype # type: ignore

(dtype('float64'), dtype('int64'))

In [23]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# check the data type
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [24]:
# データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [25]:
from torch.utils.data import TensorDataset

# 入力値と目標値を結合
dataset = TensorDataset(x, t)
dataset

In [26]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [27]:
# (入力変数, 教師データ)の形でデータが格納されている
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [28]:
type(dataset[0])

tuple

In [29]:
# 1サンプル目の入力変数
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [30]:
# 1サンプル目の目標値
dataset[0][1]

tensor(0)

In [31]:
# サイズ数の確認
len(dataset)

150

In [32]:
# 各データセットのサンプル数を決定
n_train = int(len(dataset) * 0.6)
n_valid = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_valid

# それぞれのサンプル数を確認
n_train, n_valid, n_test

(90, 30, 30)